In [53]:
import sys
!{sys.executable} -m pip install kfp >/dev/null

In [54]:
import os
import json
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kfp import components
from kubernetes import client as k8s_client

In [55]:
dkube_training_op           = components.load_component_from_file("/mnt/dkube/pipeline/components/training/component.yaml")
dkube_serving_op            = components.load_component_from_file("/mnt/dkube/pipeline/components/serving/component.yaml")

In [56]:
image = "ocdr/d3-datascience-pytorch-cpu:v1.6"
serving_image = "ocdr/pytorchserver:1.6"
training_program = "Img-DN"
dataset = "Img-DN"
model = "Img-DN"
training_script = "python model-care.py"
transformer_code="image-denoising/examples/transformer.py"
user = os.getenv('USERNAME')
framework = "pytorch"
f_version =  "1.6"
output_mount_point = "/opt/dkube/output/"
input_mount_point = "/opt/dkube/input/"

In [57]:
@kfp.dsl.pipeline(
    name='Img-Denoising',
    description='IMAGE-DENOISING EXAMPLE'
)
def image_denoising_pipeline(token):
    
        train       = dkube_training_op(token, json.dumps({"image": image}),
                                    framework=framework, version=f_version,
                                    program=training_program, run_script=training_script,
                                    datasets=json.dumps([dataset]), outputs=json.dumps([model]),
                                    input_dataset_mounts=json.dumps([input_mount_point]),
                                    output_mounts=json.dumps([output_mount_point]))

       
        serving     = dkube_serving_op(token, train.outputs['artifact'], device='cpu', 
                                    serving_image=json.dumps({"image": serving_image}),
                                    transformer_image=json.dumps({"image": image}),
                                    transformer_project=training_program,
                                    transformer_code=transformer_code).after(train)

### Create a Run

In [58]:
existing_token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=existing_token)
client.create_run_from_pipeline_func(image_denoising_pipeline, arguments={"token":existing_token})

RunPipelineResult(run_id=0b23b4ce-4b72-4bc5-b653-a34cb7f4ebc0)

### Compile and generate tar ball

In [59]:
compiler.Compiler().compile(image_denoising_pipeline, 'dkube_denoising_pl.tar.gz')

### Upload pipeline

In [60]:
existing_token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=existing_token)
try:
    client.upload_pipeline(pipeline_package_path = 'dkube_denoising_pl.tar.gz', pipeline_name = 'Img-Denoising', description = None)
except BaseException as e:
    print(e)